https://codereview.stackexchange.com/questions/191218/go-board-game-check-for-removed-stones-in-python/191262?noredirect=1#comment367158_191262

# Original

In [1]:
import numpy as np

In [2]:
board = np.array([
    [ 1, -1,  0,  0, -1,  1,  0],
    [-1,  0,  0,  0, -1,  1,  1],
    [ 0,  0,  0,  1, -1,  1,  1],
    [ 0,  0,  1, -1,  1, -1, -1],
    [ 0,  0,  0,  1,  0,  0, -1],
    [ 0,  0,  0,  0,  0, -1,  1],
    [ 0,  0,  0,  0,  0,  0,  1],
])

In [3]:
from enum import IntEnum
class Position(IntEnum):
    black = -1
    empty = 0
    white = 1
class Liberty(IntEnum):
    taken = -9
    opponent = -1
    unknown = 0
    free = 1

In [4]:
white_board = (board== Position.white)
black_board = (board== Position.black)
has_stone = np.logical_or(black_board,white_board)

In [5]:
def floodfill(liberties,y,x):
    """
    flood fill a region that is now known to have liberties. 1.0 signals a liberty, 0.0 signals
    undecided and -1.0 a known non-liberty (black stone)

    liberties is an np.array of currently known liberties and non-liberties
    """

    #"hidden" stop clause - not reinvoking for "liberty" or "non-liberty", only for "unknown".
    if liberties[y][x] == Liberty.unknown:  
        liberties[y][x] = Liberty.free
        if y > 0:
            floodfill(liberties,y-1,x)
        if y < liberties.shape[0] - 1:
            floodfill(liberties,y+1,x)
        if x > 0:
            floodfill(liberties,y,x-1)
        if x < liberties.shape[1] - 1:
            floodfill(liberties,y,x+1)

In [6]:
def capture_pieces(black_board, white_board):
    """Remove all pieces from the board that have 
    no liberties. This function modifies the input variables in place.

    black_board is a 19x19 np.array with value 1.0 if a black stone is
    present and 0.0 otherwise.

    white_board is a 19x19 np.array similar to black_board.

    """

    has_stone = np.logical_or(black_board,white_board)
    white_liberties = np.ones_like(black_board, dtype='int8') * Liberty.unknown
    black_liberties = np.ones_like(white_board, dtype='int8') * Liberty.unknown

    # stones in opposite color have no liberties
    white_liberties[black_board] = Liberty.opponent
    black_liberties[white_board] = Liberty.opponent
#     print(list(zip(*np.where(~has_stone))))
    for y, x in zip(*np.where(~has_stone)):
        floodfill(white_liberties,y,x)
        floodfill(black_liberties,y,x)
        

#     for y in range(has_stone.shape[0]):
#         for x in range(has_stone.shape[1]):
#             if not has_stone[y,x]:

    white_liberties[white_liberties == Liberty.unknown] = Liberty.taken
    black_liberties[black_liberties == Liberty.unknown] = Liberty.taken
#     print(white_liberties)

    white_board = np.where(white_liberties == Liberty.taken, False, white_board)
    black_board = np.where(black_liberties == Liberty.taken, False, black_board)
#     black_board[black_liberties == Liberty.taken] = False
    return black_board, white_board

In [7]:
white_board = (board== Position.white)
black_board = (board== Position.black)
%timeit  capture_pieces(black_board, white_board)

2.86 ms ± 215 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
black_original, white_original = capture_pieces(black_board, white_board)

# My original attempt

In [9]:
def capture_coloured_pieces(board: np.array, colour: Position):
    '''
    takes a `board` and returns the captured stones of `colour`
    '''
    liberties = np.ones_like(board) * Liberty.unknown
    liberties[board == -colour] = Liberty.opponent
    liberties[board == Position.empty] = Liberty.free
    empty = set(zip(*np.where(board == Position.empty)))
    stones = set(zip(*np.where(board == colour)))
    while empty:
        start = empty.pop()
        # print(f'starting at {start}')
        floodfill_set(liberties, start, empty, stones)
#     b1 = board.copy()
#     for x, y in stones:
#         b1[x, y] = Liberty.taken
    b1 = board.copy()
    stone_coords = list(zip(*stones))
    b1[stone_coords] = Position.empty

    return b1 == colour
#     return stones, liberties, b1


In [10]:
def floodfill_set(liberties, coord, empty, stones):
    x, y = coord
#     print(f'test {coord},{ liberties[x, y]}')
    empty.discard(coord)
    stones.discard(coord)
    liberties[x, y] = Liberty.free
    coords = ((x, y-1), (x, y+1), (x-1, y), (x+1, y))

    for coord in coords:
        if coord in (empty | stones):
            floodfill_set(liberties, coord, empty, stones)

In [11]:
assert np.array_equal(black_original, capture_coloured_pieces(board, Position.black))
assert np.array_equal(white_original, capture_coloured_pieces(board, Position.white))

In [12]:
%timeit capture_coloured_pieces(board, Position.black), capture_coloured_pieces(board, Position.white)

444 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# only set


In [13]:
def capture_coloured_pieces2(board: np.array, colour: Position):
    '''
    takes a `board` and returns the captured stones of `colour`
    '''
    empty = set(zip(*np.where(board == Position.empty)))
    stones = set(zip(*np.where(board == colour)))
    while empty:
        start = empty.pop()
        # print(f'starting at {start}')
        floodfill_set2(start, empty, stones)
    b1 = board.copy()
    stone_coords = list(zip(*stones))
    b1[stone_coords] = Position.empty

    return b1 == colour
def floodfill_set2(coord, empty, stones):
    x, y = coord
#     print(f'test {coord},{ liberties[x, y]}')
    empty.discard(coord)
    stones.discard(coord)
    coords = ((x, y-1), (x, y+1), (x-1, y), (x+1, y))

    for coord in coords:
        if coord in (empty | stones):
            floodfill_set2(coord, empty, stones)

In [14]:
assert np.array_equal(black_original, capture_coloured_pieces2(board, Position.black))
assert np.array_equal(white_original, capture_coloured_pieces2(board, Position.white))

In [15]:
%timeit capture_coloured_pieces2(board, Position.black), capture_coloured_pieces2(board, Position.white)

368 µs ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Convolution

In [16]:
from scipy.signal import fftconvolve

In [17]:
kernel = np.array([
    [0, 1, 0],
    [1, 1, 1],
    [0, 1, 0],
])

In [51]:
def capture_colour(board, color):
    sentinel = 1000
    b1 = board.copy()
    b1[board == Position.empty] = sentinel
    b1[board == -color] = 0
    b1[board == color] = 1
#     print(b1)
    ret = fftconvolve(b1, kernel, mode='same')
#     print(ret)
    ret[board == -color] = -color
    ret[board == Position.empty] = Position.empty
    ret[(board == color) & (ret < sentinel)] = color
    ret[(board == color) & (ret >= sentinel)] = Position.empty
    return ret.astype(int)

In [54]:
def capture_all(board, colour):
    board1, board2 = capture_colour(board, colour), board
    c = 0
    while not np.array_equal(board1, board2):
#         print(f'iteration {c}')
        c += 1
        board1, board2 = capture_colour(board1, colour), board1
#     print(board1)
    return (board == colour) & (board != board1)

In [26]:
board

array([[ 1, -1,  0,  0, -1,  1,  0],
       [-1,  0,  0,  0, -1,  1,  1],
       [ 0,  0,  0,  1, -1,  1,  1],
       [ 0,  0,  1, -1,  1, -1, -1],
       [ 0,  0,  0,  1,  0,  0, -1],
       [ 0,  0,  0,  0,  0, -1,  1],
       [ 0,  0,  0,  0,  0,  0,  1]])

In [55]:
black_original, capture_all(board, Position.black)

(array([[False,  True, False, False,  True, False, False],
        [ True, False, False, False,  True, False, False],
        [False, False, False, False,  True, False, False],
        [False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True],
        [False, False, False, False, False,  True, False],
        [False, False, False, False, False, False, False]]),
 array([[False,  True, False, False,  True, False, False],
        [ True, False, False, False,  True, False, False],
        [False, False, False, False,  True, False, False],
        [False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False,  True],
        [False, False, False, False, False,  True, False],
        [False, False, False, False, False, False, False]]))

In [56]:
assert np.array_equal(black_original, capture_all(board, Position.black))
assert np.array_equal(white_original, capture_all(board, Position.white))

In [57]:
%timeit capture_all(board, Position.white), capture_all(board, Position.black)

1.29 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
